# ArUco Board Generation and Detection

This notebook demonstrates how to:

1. Build a custom ArUco board.
2. Detect the ArUco markers.
3. Reproject the photo using the markers found (Pose Estimation).


In [ ]:
import sys

import cv2
from loguru import logger as lg
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Configure logger
lg.remove()
lg.add(sys.stderr, format="{time} {level} {message}", level="INFO")

lg.info(f"OpenCV Version: {cv2.__version__}")

In [ ]:
# Define ArUco Dictionary and Parameters
# We will use a predefined dictionary: DICT_6X6_250
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
aruco_params = cv2.aruco.DetectorParameters()

lg.info("ArUco dictionary and parameters initialized.")

In [ ]:
# Generate Custom ArUco Board (GridBoard)
markers_x = 5
markers_y = 7
marker_length = 100  # pixels (or arbitrary units for generation)
marker_separation = 10  # pixels

# Create the board
board = cv2.aruco.GridBoard(
    size=(markers_x, markers_y),
    markerLength=marker_length,
    markerSeparation=marker_separation,
    dictionary=aruco_dict,
)

# Generate the image of the board
# Size of the output image in pixels.
# Should be large enough to hold the board.
# Width = markers_x * marker_length + (markers_x - 1) * marker_separation + margins
# Height = markers_y * marker_length + (markers_y - 1) * marker_separation + margins
img_width = 600
img_height = 800
margin = 20
border_bits = 1

board_image = board.generateImage(
    (img_width, img_height),
    None,
    margin,
    border_bits,
)

# Save the board image
board_filename = "custom_aruco_board.png"
cv2.imwrite(board_filename, board_image)
lg.info(f"Board image saved to {board_filename}")

# Display the board
plt.figure(figsize=(8, 10))
plt.imshow(board_image, cmap="gray")
plt.title("Generated ArUco GridBoard")
plt.axis("off")
plt.show()

In [ ]:
# Load Input Image (Simulating a photo with the generated board)
# In a real application, this would be a photo from a camera.
input_image = cv2.imread(board_filename)

if input_image is None:
    lg.error(f"Could not load image {board_filename}")
else:
    lg.info(f"Loaded image {board_filename} with shape {input_image.shape}")

    # Convert to RGB for matplotlib visualization
    input_image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 10))
    plt.imshow(input_image_rgb)
    plt.title("Input Image")
    plt.axis("off")
    plt.show()

In [ ]:
# Detect ArUco Markers
# Using ArucoDetector class if available (OpenCV 4.7+),
# otherwise fallback to detectMarkers function

try:
    detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)
    corners, ids, rejected = detector.detectMarkers(input_image)
    lg.info("Used ArucoDetector class.")
except AttributeError:
    lg.info("ArucoDetector class not found, using detectMarkers function.")
    corners, ids, rejected = cv2.aruco.detectMarkers(  # pyright: ignore[reportAttributeAccessIssue]
        input_image, aruco_dict, parameters=aruco_params
    )

if ids is not None:
    lg.info(f"Detected {len(ids)} markers.")
else:
    lg.warning("No markers detected.")

In [ ]:
# Visualize Detected Markers
output_image = input_image.copy()

if ids is not None:
    cv2.aruco.drawDetectedMarkers(output_image, corners, ids)

    # Convert to RGB for display
    output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 10))
    plt.imshow(output_image_rgb)
    plt.title("Detected Markers")
    plt.axis("off")
    plt.show()
else:
    lg.warning("No markers to visualize.")

In [ ]:
# Estimate Pose and Reproject Axis

# Define dummy camera matrix and distortion coefficients
# In a real scenario, these should be obtained via camera calibration
focal_length = img_width
center = (img_width / 2, img_height / 2)
camera_matrix = np.array(
    [[focal_length, 0, center[0]], [0, focal_length, center[1]], [0, 0, 1]], dtype=float
)
dist_coeffs = np.zeros((5, 1))

# Estimate Board Pose
if ids is not None and len(ids) > 0:
    # Match image points
    obj_points, img_points = board.matchImagePoints(corners, ids)

    if obj_points is not None and len(obj_points) > 0:
        # Solve PnP to find board pose
        retval, rvec, tvec = cv2.solvePnP(
            obj_points, img_points, camera_matrix, dist_coeffs
        )

        if retval:
            lg.info("Board pose estimated.")
            lg.info(f"Rotation Vector:\n{rvec}")
            lg.info(f"Translation Vector:\n{tvec}")

            # Draw the axis for the board
            # The axis length should be comparable to marker length or board size
            axis_length = marker_length * 3

            pose_image = output_image.copy()
            cv2.drawFrameAxes(
                pose_image, camera_matrix, dist_coeffs, rvec, tvec, axis_length
            )

            # Convert to RGB for display
            pose_image_rgb = cv2.cvtColor(pose_image, cv2.COLOR_BGR2RGB)

            plt.figure(figsize=(8, 10))
            plt.imshow(pose_image_rgb)
            plt.title("Pose Estimation (Board Axis)")
            plt.axis("off")
            plt.show()
        else:
            lg.warning("Failed to estimate board pose.")
    else:
        lg.warning("Not enough points to estimate board pose.")
else:
    lg.warning("No markers detected for pose estimation.")

In [ ]:
# Perspective Correction (Rectify the Board)
# Use the detected markers to warp the image so the board is "squared" and flat.

if ids is not None and len(ids) > 0:
    # Re-match points to ensure we have the data
    obj_points, img_points = board.matchImagePoints(corners, ids)

    min_obj_points = 4
    if obj_points is not None and len(obj_points) >= min_obj_points:
        # 1. Prepare Source Points (from image)
        src_points = img_points.reshape(-1, 2)

        # 2. Prepare Destination Points (from board definition)
        # obj_points are in 3D (X, Y, Z). For GridBoard, Z=0.
        # We need to map these to 2D pixel coordinates for the output image.
        # GridBoard coordinate system: X right, Y up (usually), Z out.
        # Image coordinate system: X right, Y down.

        object_points_2d = obj_points.reshape(-1, 3)[:, :2]  # Drop Z

        # Calculate bounds to determine output image size and offsets
        min_x = np.min(object_points_2d[:, 0])
        max_x = np.max(object_points_2d[:, 0])
        min_y = np.min(object_points_2d[:, 1])
        max_y = np.max(object_points_2d[:, 1])

        board_width = max_x - min_x
        board_height = max_y - min_y

        # Define a margin for the output image
        rect_margin = 50

        # Destination points in the output image
        # We map:
        # X -> X - min_x + margin
        # Y -> (max_y - Y) + margin  <-- Flip Y so top of board (max_y)
        #                                is at top of image (low y)

        dst_points = np.zeros_like(object_points_2d)
        dst_points[:, 0] = object_points_2d[:, 0] - min_x + rect_margin
        dst_points[:, 1] = (max_y - object_points_2d[:, 1]) + rect_margin

        # Output image size
        out_width = int(board_width + 2 * rect_margin)
        out_height = int(board_height + 2 * rect_margin)

        # 3. Compute Homography
        H, mask = cv2.findHomography(src_points, dst_points)

        # 4. Warp Perspective
        rectified_image = cv2.warpPerspective(input_image, H, (out_width, out_height))

        # Display
        rectified_image_rgb = cv2.cvtColor(rectified_image, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(8, 10))
        plt.imshow(rectified_image_rgb)
        plt.title("Rectified (Squared) Image")
        plt.axis("off")
        plt.show()

        lg.info(f"Image rectified to size {out_width}x{out_height}")
    else:
        lg.warning("Not enough points to rectify image.")
else:
    lg.warning("No markers detected for rectification.")